## Data Generating Process:

The population regression equation is $y = F w + \epsilon$. In the following simulations we assume $y \in \mathbb{R}^n$, $F \in \mathbb{R}^{n \times d}$ so naturally $w \in \mathbb{R}^d$ and $\epsilon \in \mathbb{R}^n$. 

Let us further assume that elements in $\epsilon = (\epsilon_1,\ldots, \epsilon_n)$ are i.i.d. with the Gaussian distribution of $\mathcal{N}(0, \sigma^2_{\text{noise}})$.

We also generate the elements of the design matrix $F$ by i.i.d. draws from $\mathcal{N}(0,1)$, that is $F_{i,j} \sim \mathcal{N}(0,1)$ where $i\in[n]$ and $j\in [d]$. Subsequently, we *standardize* the columns of the factor matrix $F$.

Lastly, we need to generate the *true factor loadings*, i.e., $w = (w_1,\ldots,w_d)$. We pick a fraction of indices from $1$ to $d$ uniformly at random (called the *active* set). The size of this fraction is determined by the *sparsity level* chosen in $[0,1]$. These indices represent the non-zero values in $w$. If an index $i$ belongs to this active set of indices, then we independently draw $w_i \sim \mathcal{N}(0,1)$. Otherwise $i$ represents an inactive loading, and hence we independently draw $w_i \sim \mathcal{N}(0,\sigma^2_{\text{off}})$, where $\sigma_{\text{off}}\ll 1$ is rather small, say $0.01$.

In [1]:
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
class DGP:
    
    """
    
    Data generating process class
    
    Parameters
    -------------
        n : number of observations
        d : total number of factors
        sparsity : fraction of factor loadings that are non-zero
        sigmaNoise : Std of the additive i.i.d. Gaussian noise 
        sigmaInactive : Std of the inactive elements of factor loadings
        
        
    Methods
    ------------
        sampleNoise : Generating the vector of additive noise, epsilon
        sampleFactors : Generating the factor matrix, F
        sampleW : Generating the factor loadings, w
        generateY : Using the above methods, it generates the outcome y
        
        
    Attributes
    ------------
        All the parameters plus
        noise : returning the sampled noise vector
        F : returning the sampled factor matrix
        w :  returning the factor loadings
        y : returning the outcome vector y
        
    """
    
    def __init__(self, n=10, d=100, sparsity=0.1, sigmaNoise=0.05, sigmaInactive=0.002):
        self.n = n
        self.d = d
        self.sparsity = sparsity
        self.sigmaNoise = sigmaNoise
        self.sigmaInactive = sigmaInactive
        
        
    def sampleNoise(self, seedNum):
        self.noise = norm.rvs(loc=0, scale=self.sigmaNoise, size=self.n, random_state=seedNum)
        
        
    def sampleFactors(self, seedNum):
        F = norm.rvs(loc=0, scale=1, size=(self.n, self.d), random_state=seedNum)
        
        # Standardizing the columns of F:
        scaler = StandardScaler()
        self.F = scaler.fit_transform(F)
        

    
    def sampleW(self, seedNum):
        np.random.seed(seedNum)
        
        # Finding the size of non-zero elements of w:
        activeSize = int(self.d * self.sparsity)
        
        # Drawing the active and inactive indices uniformly at random from d positions:
        active = np.random.choice(range(self.d), size=activeSize, replace=False)
        in_active = np.array(list((set(range(self.d)) - set(active))))
        
        # Creating the factor loadings:
        w = np.zeros(self.d)
        w[active] = norm.rvs(loc=0, scale=1, size=activeSize)
        w[in_active] = norm.rvs(loc=0, scale=self.sigmaInactive, size=self.d - activeSize)
        self.w = w

        
    def generateY(self, *args):
        if len(args) == 3:
            F, w, eps = args
            self.y = F @ w + eps
        else:
            self.y = (self.F @ self.w) + self.noise

## Smallest $\ell_0$ norm extraction:

We want to find the vector of weights with the smallest $\ell_0$ norm, accepting a certain level of $\ell_2$ training error on the observation vector:


$$
\normalsize
\min_w \lVert w \rVert_0\, \text{ subject to }\, \frac{1}{n}\lVert y - F w \rVert_2^2 \leq b^2\,.
$$
In this problem, $\lVert \cdot \rVert_0$ is the $\ell_0$ norm, and $b$ is the $\ell_2$ slack. This is essentially a combinatorial problem, because the $\lVert \cdot \rVert_0$ norm counts the number of non-zero elements in $w$. Instead, we use a *differentiable surrogate* for this term:

$$
\normalsize
\lVert w \rVert_0 \approx S(w):= d - \sum_{k=1}^d e^{-w^2_k/2\tau^2}
$$
As $\tau$ decreases down to $0$, the approximation becomes more precise. Therefore, fixing a certain level $\tau$ we solve the following (non-convex) minimization instead of the above program:

$$
\normalsize
\min_{w} S(w)\, \text{ subject to }\, \frac{1}{n}\lVert y - F w \rVert_2^2 \leq b^2\,.
$$

The gradient of the above objective function with respect to $w$ is

$$
\normalsize
\nabla S(w) =  \frac{1}{\tau^2} \left[w_1 e^{-w^2_1/2\tau^2}, \ldots,w_d e^{-w^2_d/2\tau^2} \right]^T\,.
$$

## Optimization algorithm:

### 1. Projection onto the feasible set (inner loop):

Suppose we have a vector $w$, and we seek to project it onto the feasible set $\mathcal{F(b)}=\left\{v: \frac{1}{n}\lVert y - F v \rVert_2^2 \leq b^2\right\}$. For this, one needs to solve the following convex minimizatin problem:
$$
\normalsize
\min \lVert w - v\rVert_2^2\, \text{ subject to } \lVert y - F v \rVert_2^2 \leq  b^2 n\,.
$$

The associated Largrangian for this problem is
$$
\normalsize
\mathcal{L}(v, \lambda) = \lVert w - v\rVert_2^2 + \lambda \left(\lVert y - F v \rVert_2^2 - b^2 n\right)\,.
$$

At the optimum $(v^*, \lambda^*)$, the necessary first order condition with respect to the primal variable $v$ is

$$
\normalsize
\nabla_v \mathcal{L}(v^*, \lambda^*) = 0\, \Rightarrow v^* = (I + \lambda^* F^T F)^{-1}(w + \lambda^* F^T y)\,.
$$

The dual function is obtained by plugging the above form of $v$ into the Largrangian $\mathcal{L}(v, \lambda)$. Let us denote the dual function by $\lambda \mapsto \Phi(\lambda)$:

$$
\normalsize
\Phi(\lambda) = -(w + \lambda F^T y)^T (I + \lambda F^T F)^{-1} (w + \lambda F^T y) + \lVert w \rVert_2^2 +\lambda (\lVert y \rVert_2^2 - b^2 n)\,.
$$

Consequently, the dual problem is to maximize $\Phi(\lambda)$ on $\lambda \geq 0$. Since the *Slater's* condition holds, then one has strong duality, and the optimal value for the dual problem matches the optimal value of the primal problem. 

We need to find the $\lambda^*$ that minimizes $-\Phi(\lambda)$ on the region $\lambda^*\geq 0$, that in turn gives us the optimal dual variable. Substituting that into the expression for $v^*$ gives the projection of $w$ onto $\mathcal{F}(b)$. Minimizing $-\Phi(\lambda)$ is *equivalent* to

$$
\normalsize
\min J(\lambda)\, \text{ on } \lambda \geq 0\,, \text{where } J(\lambda):=(w + \lambda F^T y)^T (I + \lambda F^T F)^{-1} (w + \lambda F^T y) - \lambda (\lVert y \rVert_2^2 - b^2 n)\,.
$$

Using a *variational* method, we can find $J'(\lambda)$:

$$
\normalsize
J'(\lambda) = -(w + \lambda F^T y)^T (I + \lambda F^T F)^{-1}(F^T F)(I + \lambda F^T F)^{-1}(w + \lambda F^T y)+2y^TF (I + \lambda F^T F)^{-1}(w + \lambda F^T y) - \lVert y \rVert_2^2 + b^2 n\,.
$$

Given the above information, one can apply Gradient Descent with line search and backtracking to find $\lambda^*$ and subsequently the projection of $w$ denoted by $w^+$. The projection algorithm is as follows:

- initialize $\lambda = \lVert w \rVert_2^2 / \lVert y \rVert_2^2$ and find $J'(\lambda)$
    
- while $\lvert J'(\lambda) \rvert / J(\lambda) > 0.01$:
    - Set the step size $t=1$, and find $J'(\lambda)$
    - *Line search + backtracking* with parameters $\alpha_{\text{dual}} \in (0.01,0.3)$ and $\beta_{\text{dual}} \in (0.1,0.8)$:
    - while $J\left(\lambda - t J'(\lambda)\right) > J(\lambda) - \alpha_{\text{dual}} t J'(\lambda)^2$:
        - update $t \leftarrow \beta_{\text{dual}} t$ (i.e., depreciating step size at the rate of $\beta_{\text{dual}}$)
    - update $\lambda \leftarrow \lambda - t J'(\lambda)$
    
    
### 2. Optimizing the weights (outer loop):
Now that we know how to optimally find the projection of a weight vector $w$ onto $\mathcal{F}(b)$, we can explain the descent algorithm behind the $\ell_0$ norm minimization:

- initialize $w = F^{\dagger}y$ and $\tau = 2\max_{k} |w_k|$
- while $\tau > \tau_{\text{min}}$:
        
    - Repeat $T$ times:
        - update $w \leftarrow w - \mu \tau^2 \nabla S(w)$
        - project $w$ onto the feasible set $\mathcal{F}(b)$, hence $w \leftarrow w^+$
    - shrink $\tau$ by $\tau \leftarrow r \tau$ for $r \in (0,1)$
    
### 3. In reality $b$ is not known:
Observe that the prior knowledge about $b$ is not always available. All we have access to is the factor matrix $F$ and the outcomes $y$. Therefore, to go about this problem, in a recursive manner, we estimate $w$, and based on that choose a new $b$ until a convergence criterion is reached. Initially $w = F^{\dagger}y$. Therefore, we set $b = \lVert y - F w\rVert_2 * \xi$, where $\xi>1$ is a hyperparameter that makes sure the noise is not trained. Based on this $b$, we find the new $w$ and repeat this process until convergence.

In [3]:
class optimize:
    """
    Implementation of the explained algorithm (inner and outer loop)
    
    Parameters
    ----------
        tauMin : minimum tau in the sequence of surrogate functions
        T : the number of iterations for every fixed tau
        mu : the multiplier coefficient in the gradient descent of outer loop
        r : depreciation rate of tau
        alpha : first parameter of the backtracking line seach in the inner loop
        beta : second parameter of the backtracking line seach in the inner loop

        
    Attributes
    ----------
        wHat : estimated w 
        
    
    """
    def __init__(self, tauMin=0.005, T=5, mu=2, r=0.9, alpha=0.1, beta=0.9, xi=1.4):
        self.tauMin = tauMin
        self.T = T
        self.mu = mu
        self.r = r
        self.alpha = alpha
        self.beta = beta
        self.xi = xi
    #--------------------------------------------------------------------------------------------------    
        
        
    ### Implemeting the projection step (inner loop):   
    
    def dual(self, F, y, lam, w, b):
        # this is the (negative) of the dual function
        n, d = F.shape  
        vec = w + lam * F.T @ y
    
        return np.dot(vec, np.linalg.inv(np.eye(d) + lam * F.T @ F) @ vec) - lam * (np.dot(y, y) - b**2 * n)
    
    def dual_gradient(self, F, y, lam, w, b):
        # this method computes the gradient of dual function
        n, d = F.shape 
        vec = w + lam * F.T @ y
        tmp = np.linalg.inv(np.eye(d) + lam * F.T @ F) 
        
        return -np.dot(vec, tmp @ F.T @ F @ tmp @ vec) + 2 * y.T @ F @ tmp @ vec - np.dot(y, y) + b**2 * n
    
    def projection(self, F, y, w, b):
        # This function performs the projection onto the feasible region
        # First, we find the optimal dual variable lambda:
        d = F.shape[1]
        lam = np.dot(w, w) / np.dot(y, y)
        while np.abs(self.dual_gradient(F, y, lam, w, b)) / self.dual(F, y, lam, w, b) > 0.1:
            t = 1
            dual = self.dual(F, y, lam, w, b)
            dualGrad = self.dual_gradient(F, y, lam, w, b)
            while self.dual(F, y, lam - t * dualGrad, w, b) > dual - self.alpha * t * dualGrad**2:
                t *= self.beta
            lam -= t * dualGrad
            
        # Next, we find the projection of weight vector w onto the feasible region (having found lambda):
        return np.linalg.inv(np.eye(d) + lam * F.T @ F) @ (w + lam * F.T @ y)
    #----------------------------------------------------------------------------------------
            
    ### Optimizing the weights (outer loop):
    
    def surrogateGradient(self, w, tau):
        # We intentionally do NOT divide by tau^2, because in all future cases, \
        # the gradient vector is multiplied by this constant.

        return w * np.exp(-w**2 / (2 * tau**2))
    
    def fit(self, F, y):
        # Initializing w and b
        n = y.shape[0]
        w = np.linalg.pinv(F) @ y
        b_prev = np.sqrt(np.sum((y - F @ w)**2) / n) * self.xi
        b_curr = b_prev * 1.15
        
        while np.abs(b_curr - b_prev) / b_prev > 0.2:
            tau = 2 * np.max(np.abs(w))
            while tau > self.tauMin:
                for _ in range(self.T):
                    w -= self.mu * self.surrogateGradient(w, tau)
                    w = self.projection(F, y, w, b_curr)
                tau *= self.r 
            b_prev, b_curr = b_curr, np.sqrt(np.sum((y - F @ w)**2) / n) * self.xi
            
        self.wHat = w
        